In [3]:
#action 1
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly,KNNBasic
from surprise import accuracy
from surprise.model_selection import KFold


In [9]:
#读取数据
reader = Reader(line_format="user item rating timestamp",sep=",",skip_lines=1)
data = Dataset.load_from_file("./ratings.csv",reader=reader)
train_set = data.build_full_trainset()

In [15]:
#SGD优化
bsl_options = {"option":"sgd","n_epochs":5}
algo = BaselineOnly(bsl_options=bsl_options)
kf = KFold(n_splits=3)
for trainset,testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions,verbose=True)
    uid = str(196)
    lid = str(302)
    pred = algo.predict(uid,lid,r_ui=4,verbose=True)


Estimating biases using als...
RMSE: 0.8646
user: 196        item: 302        r_ui = 4.00   est = 4.07   {'was_impossible': False}
Estimating biases using als...
RMSE: 0.8686
user: 196        item: 302        r_ui = 4.00   est = 4.05   {'was_impossible': False}
Estimating biases using als...
RMSE: 0.8649
user: 196        item: 302        r_ui = 4.00   est = 4.11   {'was_impossible': False}


In [ ]:
#action 2


In [17]:
grammar = '''
战斗 => 施法  ， 结果 。
施法 => 主语 动作 技能 
结果 => 主语 获得 效果
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉
动作 => 施放 | 使用 | 召唤 
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得 
效果 => 数值 状态
数值 => 1 | 1000 |5000 | 100 
状态 => 法力 | 生命
'''

In [7]:
#action 3
import random
def getGrammarDict(gram,linesplit="\n",gramsplit="=>"):
    #定义字典
    result = {}
    for line in gram.split(linesplit):
        #去掉首位空格，如果危机哦那个则推出
        if not line.strip():
            continue
        expr,statement = line.split(gramsplit)
        result[expr.strip()]=[i.split() for i in statement.split("|")]
    return result


In [10]:
#生成句子
def generate(gramdict,target,isEeng=False):
    if target not in gramdict:
        return target
    find = random.choice(gramdict[target])
    blank = ""
    if isEeng:
        blank =''
    return blank.join(generate(gramdict,t,isEeng) for t in find)
gramdict = getGrammarDict(grammar)
print(generate(gramdict,"战斗"))
print(generate(gramdict,"战斗",True))
      
      

关羽使用单刀赴会，黄忠损失100法力。
许褚施放惊雷之龙，许褚损失1法力。


#thinkng 1 ALS都有哪些应用场景?
用于电影推荐，电影推荐，商品推荐等。
原理就是给各个指标参数，判定等加权重，然后将这些训练集输入als，内部使用矩阵相乘，根据这些权重，给用户对未知，未点击的商品一个分数，
就是喜好程度，然后根据按照喜好程度推荐给用户。


#thingk 2 ALS进行矩阵分解的时候，为什么可以并行化处理?
因为als使用交替最小二乘法进行求解。先固定一个参数x，然后再求矩阵y。然后再固定一个矩阵Y,求解矩阵X。
 在求解的过程中，比如固定Y，求解X的时候，X的每一行都是独立求解，这样得话，对于每一步来说，X,Y的行和列都是可以独立计算并求解的，这样就实现了并行化处理。

#thinking3 梯度下降法中的批量梯度下降（BGD），随机梯度下降（SGD），和小批量梯度下降有什么区别（MBGD)
批量梯度下降：每次迭代更新所有的样本，利用矩阵进行操作，利用矩阵进行计算，实现了并行，训练的速度很慢。
随机梯度下降：每次迭代更新一个样本，训练的速度最快，最快收敛，准确度下降。
小批量梯度下降：是对梯度下降和批量梯度下降一个折中的方法，每一个批量可以大大减小收敛所需要的迭代次数，可以收敛到结果更加接近梯度下降的结果。速度适中


#thinking 4 你阅读过和推荐系统/计算广告/预测相关的论文么？
毕业工作几年，几乎没有读过论文，近一周读了两篇。1.基于矩阵分解的协同过滤算法的并行化研究与是实现 2.基于协同过滤的个性化新闻推荐的研究和实现